### Make top-level imports

* PM4Py
* OrdinoR (*note: from local repo*)

In [4]:
from os.path import join as path_join

import pandas as pd
import pm4py

import ordinor.constants as const
from ordinor.io import read_csv
from ordinor.utils.log_preprocessing import append_case_duration

### Import original event log data file

In [5]:
DIRPATH = './data/DATA_csv'
LOGNAME = 'bpic2017'

fn = path_join(DIRPATH, f'{LOGNAME}.csv')
print(f'Import source event log {fn}')

try:
    log = read_csv(fn)
except Exception as e:
    print(e)

print(sorted(log.columns))

Import source event log ./data/DATA_csv/bpic2017.csv
Importing from CSV file ./data/DATA_csv/bpic2017.csv
Scanned 1202267 events from "./data/DATA_csv/bpic2017.csv".
--------------------------------------------------------------------------------
Number of events:		1202267
Number of cases:		31509
--------------------------------------------------------------------------------
['@@case_index', '@@index', 'Accepted', 'Action', 'CreditScore', 'EventID', 'EventOrigin', 'FirstWithdrawalAmount', 'MonthlyCost', 'NumberOfTerms', 'OfferID', 'OfferedAmount', 'Selected', 'Unnamed: 0', 'case:ApplicationType', 'case:LoanGoal', 'case:RequestedAmount', 'case:concept:name', 'concept:name', 'lifecycle:transition', 'org:resource', 'time:timestamp']


### Data preprocessing (generic)

##### Preprocessing
- Keep only events that indicate completion of activities
- Derive an event attribute indicating the type of events

In [6]:
# Keep only events of completion
log = log[log['lifecycle:transition'] == 'complete']

# Derive attribute `event_type`
log['event_type'] = log[const.ACTIVITY].apply(lambda a: a[:2])

for value in sorted(log['case:LoanGoal'].unique()):
    print(value)

print('-' * 80)

for value in sorted(log['case:ApplicationType'].unique()):
    print(value)


log.to_csv(f'data/{LOGNAME}.preprocessed.csv')
log
# print(log['event_type'].unique())
# print(log['event_type'].nunique())

Boat
Business goal
Car
Caravan / Camper
Debt restructuring
Existing loan takeover
Extra spending limit
Home improvement
Motorcycle
Not speficied
Other, see explanation
Remaining debt home
Tax payments
Unknown
--------------------------------------------------------------------------------
Limit raise
New credit


,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,@@index,@@case_index,event_type
0,686058,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,A_
1,686059,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,A_
5,686063,statechange,User_1,A_Concept,Application,ApplState_385184570,complete,2016-08-03 15:58:28.299000+00:00,"Other, see explanation",New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,A_
8,686066,statechange,User_5,A_Accepted,Application,ApplState_856156982,complete,2016-08-05 13:57:07.419000+00:00,"Other, see explanation",New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,0,A_
9,686067,Created,User_5,O_Create Offer,Offer,Offer_410892064,complete,2016-08-05 13:59:57.320000+00:00,"Other, see explanation",New credit,...,22.0,True,241.28,False,0.0,5000.0,NaN,9,0,O_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202254,918287,statechange,User_113,A_Validating,Application,ApplState_306039057,complete,2016-10-19 10:28:04.832000+00:00,Caravan / Camper,New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1202254,31508,A_
1202255,918288,statechange,User_113,O_Returned,Offer,OfferState_1258740957,complete,2016-10-19 10:28:11.487000+00:00,Caravan / Camper,New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,Offer_913122697,1202255,31508,O_
1202260,918293,statechange,User_109,A_Incomplete,Application,ApplState_756087123,complete,2016-10-19 12:44:29.292000+00:00,Caravan / Camper,New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1202260,31508,A_
1202264,918297,statechange,User_68,O_Accepted,Offer,OfferState_1927291643,complete,2016-10-24 06:24:30.056000+00:00,Caravan / Camper,New credit,...,NaN,NaN,NaN,NaN,NaN,NaN,Offer_913122697,1202264,31508,O_
